In [ ]:
from IPython.display import display, HTML, Javascript
import ipywidgets as ipw
import os
import re
from ai_agent import single_agent
from src import utils
import markdown

In [ ]:
CONFIG = utils.read_json("config/config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis_aiida(
    "https://openbis-empa-dev205.ethz.ch/"
)
try:
    LLM_CONFIG = utils.read_json("/home/jovyan/api_keys/llm_config.json")
except FileNotFoundError:
    LLM_CONFIG = None

if LLM_CONFIG:
    openbis_chatbot = single_agent.OpenBISAgent(llm_config=LLM_CONFIG)

In [ ]:
increase_buttons_size = HTML(data="".join(CONFIG["save_home_buttons_settings"]))

prompt_label = ipw.HTML(
    value="<span style='font-weight: bold; font-size: 12px;'>Prompt:</span>"
)

prompt_textarea = ipw.Textarea(
    layout=ipw.Layout(width="800px", height="100px"),
    style={"description_width": "110px"},
)

enter_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Enter",
    icon="arrow-right",
    layout=ipw.Layout(width="100px", height="50px"),
)

prompt_hbox = ipw.HBox(children=[prompt_label, prompt_textarea, enter_button])

chat_label = ipw.HTML(
    value="<span style='font-weight: bold; font-size: 12px;'>Chat:</span>"
)
chat_html = ipw.HTML(
    value="<div style='border: 1px solid grey; padding: 10px; margin: 10px;'> <p><b>Agent:</b> Hi! I am openBIS chatbot and I am here to answer your questions about openBIS. How may I help you today?"
)
chat_hbox = ipw.HBox(children=[chat_label, chat_html])

quit_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Main menu",
    icon="home",
    layout=ipw.Layout(width="100px", height="50px"),
)

In [ ]:
def close_notebook(b):
    display(Javascript(data='window.location.replace("home.ipynb")'))


def log_chatbot_usage(filepath, json_data):
    input_tokens = json_data["input_tokens"]
    output_tokens = json_data["output_tokens"]

    os.makedirs("logs", exist_ok=True)
    filepath = os.path.join("logs", filepath)

    if not os.path.isfile(filepath):
        with open(filepath, "w") as f:
            f.write("input_tokens,output_tokens\n")
            f.write(f"{input_tokens},{output_tokens}\n")
    else:
        with open(filepath, "a") as f:
            f.write(f"{input_tokens},{output_tokens}\n")


def find_experiment_permid(obj_permid):
    obj = OPENBIS_SESSION.get_sample(obj_permid)
    return obj.experiment.permId


def permid_to_link(text, eln_url):
    """
    Replace permIDs in text with hyperlinks to ELN-LIMS.
    If experimentId cannot be resolved, keep permId as plain text.
    """

    pattern = r"\b\d{17}-\d+\b"

    def replacer(match):
        obj_permid = match.group(0)
        try:
            exp_permid = find_experiment_permid(obj_permid)  # may fail
            if not exp_permid:
                return obj_permid  # no experiment found
            url = (
                f"{eln_url}?menuUniqueId=%7B%22type%22:%22EXPERIMENT%22,"
                f"%22id%22:%22{exp_permid}%22%7D&viewName=showViewSamplePageFromPermId&"
                f"viewData=%7B%22permIdOrIdentifier%22:%22{obj_permid}%22,"
                f"%22paginationInfo%22:%7B%22pagOptions%22:%7B%22pageIndex%22:0,"
                f"%22pageSize%22:10,%22sortings%22:[%7B%22columnName%22:%22permId%22,"
                f"%22sortDirection%22:%22desc%22%7D],%22searchMode%22:%22GLOBAL_FILTER%22,"
                f"%22globalSearch%22:%7B%22operator%22:%22AND%22,%22text%22:null%7D,"
                f"%22searchMap%22:%7B%7D,%22columnMap%22:%7B%7D%7D,"
                f"%22currentIndex%22:0,%22previousIndex%22:null,%22totalCount%22:8%7D%7D"
            )
            return f'<a href="{url}">{obj_permid}</a>'
        except Exception:
            return obj_permid  # leave unchanged on error

    return re.sub(pattern, replacer, text)


eln_url = "https://local.openbis.ch/openbis/webapp/eln-lims/"


def ask_chatbot(change):
    prompt = prompt_textarea.value
    chat_html.value = chat_html.value + f"<p><b>User:</b> {prompt}</p>"
    agent_response = openbis_chatbot.ask_question(prompt)
    response_text = agent_response[-1].content
    response_text = permid_to_link(response_text, eln_url)
    log_chatbot_usage("chatbot_usage.csv", agent_response[-1].usage_metadata)
    # try:
    #     agent_response = openbis_agent.ask_question(prompt)
    #     response_text = agent_response[-1].content
    # except Exception as e:
    #     response_text = "Internal Server Error. Repeat your prompt."
    chat_html.value = (
        chat_html.value
        + "<p><b>Agent:</b> "
        + markdown.markdown(response_text)
        + "</p>"
    )

# openBIS chatbot

#### Note: This agent uses model Gemini Flash 2.5. The answers may contain some errors/hallucinations.

In [ ]:
display(increase_buttons_size)

if LLM_CONFIG:
    enter_button.on_click(ask_chatbot)
    display(chat_hbox)
    display(prompt_hbox)
else:
    display(
        Javascript(
            data="alert('Chatbot is not configured. Please set up the chatbot first.')"
        )
    )

display(quit_button)
quit_button.on_click(close_notebook)